# Kripton logbook

## A logbook for Krypton analysis

## Run 2980

In [ ]:
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
from __future__ import print_function
import sys
import os
from glob import glob
from time import time

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import pandas as pd
import tables as tb
import numpy as np

In [ ]:
from invisible_cities.database import load_db
from invisible_cities.core.system_of_units_c import SystemOfUnits
import invisible_cities.sierpe.blr as blr
import invisible_cities.core.mpl_functions as mpl
import invisible_cities.core.wfm_functions as wfm
import invisible_cities.core.tbl_functions as tbl
import invisible_cities.core.peak_functions_c as cpf
import invisible_cities.core.pmaps_functions as pf
import invisible_cities.core.sensor_functions as sf

In [ ]:
from scipy import signal as SGN

### Access to data base and definition of units

In [ ]:
DataPMT = load_db.DataPMT(2980)
units = SystemOfUnits()
adc_to_pes = abs(DataPMT.adc_to_pes.values)
coeff_c = abs(DataPMT.coeff_c.values)
coeff_blr = abs(DataPMT.coeff_blr.values)
DataSiPM = load_db.DataSiPM(2980)
adc_to_pes_sipm = DataSiPM.adc_to_pes.values
xs = DataSiPM.X.values
ys = DataSiPM.Y.values

### Data files

In [ ]:
RWF_file = os.environ['IC_DATA']  + '/LSC/run_2980.gdc1next.999.h5'

In [ ]:
h5rwf = tb.open_file(RWF_file,'r')

In [ ]:
pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(h5rwf)

In [ ]:
NEVT, NPMT, PMTWL = pmtrwf.shape
NEVT, NSIPM, SIPMWL = sipmrwf.shape
print("""
        Number of events in file = {}
        Number of PMTs = {}
        PMTWL = {}
        Number of SiPMs = {}
        SiPMWL = {}
      """.format(NEVT, NPMT, PMTWL,NSIPM, SIPMWL))

### Detailed analysis of event 0

In [ ]:
event = 0
RWF = pmtrwf[event]

In [ ]:
tstep = 25
signal_t = np.arange(0., PMTWL * tstep, tstep)/units.mus

In [ ]:
wfm.plot_pmt_waveforms(RWF, zoom=False, window_size=2000)

#### A double event!

In [ ]:
def plot_signal(signal, signal_start=0, signal_end=1200, ymax=200):
    """Given a series signal (t, signal), plot the signal."""
    tstep = 25
    PMTWL = signal.shape[0]
    signal_t = np.arange(0., PMTWL * tstep, tstep)/units.mus
    ax1 = plt.subplot(1, 1, 1)
    ax1.set_xlim([signal_start, signal_end])
    ax1.set_ylim([0, ymax])
    
    plt.plot(signal_t, signal)


In [ ]:
def plot_pmt_signals(signals, signal_start=0, signal_end=1200, ymax = 250):
    """Take as input a vector storing the PMT wf and plot the waveforms"""
    
    tstep = 25
    PMTWL = signals[0].shape[0]
    signal_t = np.arange(0., PMTWL * tstep, tstep)/units.mus
    plt.figure(figsize=(12, 12))
    for i in range(len(signals)):
        ax1 = plt.subplot(3, 4, i+1)
        ax1.set_xlim([signal_start, signal_end])
        ax1.set_ylim([0, ymax])
        mpl.set_plot_labels(xlabel="t (mus)",
                  ylabel="signal (pes)")
        #plt.title("EP")
        plt.plot(signal_t, signals[i])
        


### Deconvolution 

In [ ]:
event=0
t0 = time()
CWF = blr.deconv_pmt(RWF, coeff_c, coeff_blr, n_baseline=38000, thr_trigger=5)
t1 = time()        
dt = t1 - t0

print("run over  one event  in {} s".format(dt))

In [ ]:
CWFP = wfm.to_pes(CWF, adc_to_pes)

In [ ]:
tstep = 25
signal_t = np.arange(0., PMTWL * tstep, tstep)

In [ ]:
plot_pmt_signals(CWF, signal_start=0, signal_end=1200, ymax = 250)

#### Using a MAU to eliminate sinusoidal noise. 

This function is a copy of the function found in peak_functions.py, but returns the MAU, for ilustration purposes

In [ ]:
def calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=200, thr_MAU=5):
    """Compute the ZS calibrated sum of the PMTs
    after correcting the baseline with a MAU to suppress low frequency noise.
    input:
    CWF         : Corrected waveform (passed by BLR)
    adc_to_pes  : a vector with calibration constants
    n_MAU       : length of the MAU window
    thr_MAU     : treshold above MAU to select sample

    NB: This function is used mainly for testing purposes. It is
    programmed "c-style", which is not necesarily optimal in python,
    but follows the same logic that the corresponding cython function
    (in peak_functions_c), which runs faster and should be used
    instead of this one for nornal calculations.
    """

    NPMT = CWF.shape[0]
    NWF  = CWF.shape[1]
    MAU  = np.array(np.ones(n_MAU), dtype=np.double) * (1 / n_MAU)

    pmt_thr = np.zeros((NPMT, NWF), dtype=np.double)
    csum    = np.zeros(       NWF,  dtype=np.double)
    MAU_pmt = np.zeros(       NWF,  dtype=np.double)

    MAUL=[]
    for j in range(NPMT):
        # MAU for each of the PMTs, following the waveform
        MAU_pmt = SGN.lfilter(MAU, 1, CWF[j,:])
        MAUL.append(MAU_pmt)
        for k in range(NWF):
            if CWF[j,k] >= MAU_pmt[k] + thr_MAU: # >= not >. Found testing
                pmt_thr[j,k] = CWF[j,k]

    for j in range(NPMT):
        for k in range(NWF):
            csum[k] += pmt_thr[j, k] * 1 / adc_to_pes[j]
    return csum, np.array(MAUL)


####  csum computed without a MAU, csum2 computed with a MAU

In [ ]:
event=0
t0 = time()
csum = cpf.calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=100, thr_MAU=0) 
csum2, MAU = calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=100, thr_MAU=3) 
t1 = time()        
dt = t1 - t0

print("run over  one event  in {} s".format(dt))

In [ ]:
plot_pmt_signals(CWF - MAU, signal_start=0, signal_end=1200, ymax = 250)

In [ ]:
plot_pmt_signals(CWF, signal_start=0, signal_end=600, ymax = 10)

In [ ]:
plot_pmt_signals(CWF - MAU, signal_start=0, signal_end=600, ymax = 10)

In [ ]:
plot_pmt_signals(CWF, signal_start=590, signal_end=610, ymax = 200)

In [ ]:
plot_pmt_signals(CWF - MAU, signal_start=590, signal_end=610, ymax = 200)

In [ ]:
plot_signal(CWF[0], signal_start=598, signal_end=603, ymax = 50)

In [ ]:
plot_signal(CWF[0] - MAU[0], signal_start=598, signal_end=603, ymax = 50)

#### MAU introduces holes in the signal. 

In [ ]:
plot_signal(csum, signal_start=598, signal_end=603, ymax = 30)
plot_signal(csum2, signal_start=598, signal_end=603, ymax = 30)

#### But not in the csum

In [ ]:
plot_pmt_signals(CWF, signal_start=605, signal_end=820, ymax = 10)

In [ ]:
plot_pmt_signals(CWF -MAU, signal_start=605, signal_end=820, ymax = 10)

In [ ]:
plot_signal(csum, signal_start=0, signal_end=1200, ymax = 20)
plot_signal(csum2, signal_start=0, signal_end=1200, ymax = 20)

In [ ]:
plot_signal(csum, signal_start=0, signal_end=590, ymax = 5)
plot_signal(csum2, signal_start=0, signal_end=590, ymax = 5)

In [ ]:
plot_signal(csum, signal_start=597, signal_end=605, ymax = 60)
plot_signal(csum2, signal_start=597, signal_end=605, ymax = 60)

In [ ]:
plot_signal(csum, signal_start=605, signal_end=800, ymax = 2)
plot_signal(csum2, signal_start=605, signal_end=800, ymax = 2)

In [ ]:
plot_signal(csum, signal_start=820, signal_end=823, ymax = 100)
plot_signal(csum2, signal_start=820, signal_end=823, ymax = 100)

### Waveform has less than 2.5 mus

In [ ]:
plot_signal(csum, signal_start=823, signal_end=1200, ymax = 2)
plot_signal(csum2, signal_start=823, signal_end=1200, ymax = 2)

#### MAU eliminates sinusoidal noise and does not affect signal. 

In [ ]:
plot_signal(csum, signal_start=0, signal_end=590, ymax = 5)
plot_signal(csum2, signal_start=0, signal_end=590, ymax = 5)

mpl.plot_signal(signal_t/units.mus, csum, title="calibrated sum",
                signal_start=605, signal_end=690, 
                ymax = 2, t_units='mus', units="pes")
mpl.plot_signal(signal_t/units.mus, csum2, title="calibrated sum",
                signal_start=605, signal_end=690, 
                ymax = 2, t_units='mus', units="pes")

### ZS sum

#### USING MAU

In [ ]:
event = 0
t0 = time()
wfzs_ene, wfzs_indx = cpf.wfzs(csum2, threshold=0.5*units.pes)
wfzs_t = cpf.time_from_index(wfzs_indx)
t1 = time()        
dt = t1 - t0

print("run  in {} s".format(dt))

In [ ]:
mpl.plot_signal(wfzs_t/units.mus, wfzs_ene, title="calibrated sum",
                signal_start=0, signal_end=1200, 
                ymax = 10, t_units='mus', units="pes")

In [ ]:
S1 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=0,  tmax=590*units.mus, 
                 lmin=6, lmax=16, stride=4,
                 rebin=False)
S2 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=0*units.mus,  tmax=1100*units.mus, 
                 lmin=80, lmax=1000, stride=40,
                 rebin=True, rebin_stride=40)

In [ ]:
S1

In [ ]:
S2

#### Not using MAU

In [ ]:
event = 0
t0 = time()
wfzs_ene, wfzs_indx = cpf.wfzs(csum, threshold=0.5*units.pes)
wfzs_t = cpf.time_from_index(wfzs_indx)
t1 = time()        
dt = t1 - t0

print("run  in {} s".format(dt))

In [ ]:
mpl.plot_signal(wfzs_t/units.mus, wfzs_ene, title="calibrated sum",
                signal_start=0, signal_end=1200, 
                ymax = 10, t_units='mus', units="pes")

In [ ]:
S1 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=0,  tmax=590*units.mus, 
                 lmin=6, lmax=16, stride=4,
                 rebin=False)
S2 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=0*units.mus,  tmax=1100*units.mus, 
                 lmin=80, lmax=1000, stride=40,
                 rebin=True, rebin_stride=40)

In [ ]:
S1

In [ ]:
S2

### In conclussion, using a MAU seems to improve response

#### Event 1

In [ ]:
event=1
RWF = pmtrwf[event]
CWF = blr.deconv_pmt(RWF, coeff_c, coeff_blr, n_baseline=38000, thr_trigger=5)
csum = cpf.calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=100, thr_MAU=3) 
wfzs_ene, wfzs_indx = cpf.wfzs(csum, threshold=0.5*units.pes)
wfzs_t = cpf.time_from_index(wfzs_indx)
S1 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=0,  tmax=590*units.mus, 
                 lmin=6, lmax=16, stride=4,
                 rebin=False)
S2 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=0*units.mus,  tmax=1100*units.mus, 
                 lmin=80, lmax=1000, stride=40,
                 rebin=True, rebin_stride=40)

In [ ]:
CWFP = wfm.to_pes(CWF, adc_to_pes)

In [ ]:
plot_pmt_signals(CWFP, signal_start=0, signal_end=1200, ymax = 10)

In [ ]:
plot_pmt_signals(CWFP, signal_start=0, signal_end=1200, ymax = 1)

In [ ]:
plot_signal(csum, signal_start=0, signal_end=1200, ymax = 50)

In [ ]:
plot_signal(csum, signal_start=0, signal_end=590, ymax = 1)

In [ ]:
plot_signal(csum, signal_start=595, signal_end=605, ymax = 50)

In [ ]:
plot_signal(csum, signal_start=605, signal_end=1200, ymax = 1)

The average after the signal appears to be higher than before the signal. 

In [ ]:
S1

In [ ]:
S2

In [ ]:
pf.scan_s12(S2)

#### SiPMs

In [ ]:
sipm = cpf.signal_sipm(sipmrwf[event], adc_to_pes_sipm, thr=5*units.pes, n_MAU=100)
SIPM = cpf.select_sipm(sipm)
S2Si = pf.sipm_s2_dict(SIPM, S2, thr=50*units.pes)

In [ ]:
S2Si

In [ ]:
pf.plot_s2si_map(S2Si)

#### Effect of dark current

In [ ]:
sipm_i = sf.sipm_with_signal(sipmrwf[event], thr=1*16*units.adc) 

In [ ]:
sf.plot_sensor_list_ene_map(sipmrwf[event], sipm_i, stype='SiPM')

In [ ]:
sipm_i = sf.sipm_with_signal(sipmrwf[event], thr=3*16*units.adc) 
sf.plot_sensor_list_ene_map(sipmrwf[event], sipm_i, stype='SiPM')

In [ ]:
sipm_i = sf.sipm_with_signal(sipmrwf[event], thr=5*16*units.adc) 
sf.plot_sensor_list_ene_map(sipmrwf[event], sipm_i, stype='SiPM')

In [ ]:
sipm_i = sf.sipm_with_signal(sipmrwf[event], thr=7*16*units.adc) 
sf.plot_sensor_list_ene_map(sipmrwf[event], sipm_i, stype='SiPM')

In [ ]:
sipm_i = sf.sipm_with_signal(sipmrwf[event], thr=10*16*units.adc) 
sf.plot_sensor_list_ene_map(sipmrwf[event], sipm_i, stype='SiPM')

In [ ]:
sipm_i = sf.sipm_with_signal(sipmrwf[event], thr=20*16*units.adc) 
sf.plot_sensor_list_ene_map(sipmrwf[event], sipm_i, stype='SiPM')

In [ ]:
event=2
RWF = pmtrwf[event]
CWF = blr.deconv_pmt(RWF, coeff_c, coeff_blr, n_baseline=38000, thr_trigger=5)
csum = cpf.calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=100, thr_MAU=3) 
wfzs_ene, wfzs_indx = cpf.wfzs(csum, threshold=0.5*units.pes)
wfzs_t = cpf.time_from_index(wfzs_indx)
S1 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=0,  tmax=590*units.mus, 
                 lmin=6, lmax=16, stride=4,
                 rebin=False)
S2 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=0*units.mus,  tmax=1100*units.mus, 
                 lmin=50, lmax=1000, stride=40,
                 rebin=True, rebin_stride=40)
sipm = cpf.signal_sipm(sipmrwf[event], adc_to_pes_sipm, thr=5*units.pes, n_MAU=100)
SIPM = cpf.select_sipm(sipm)
S2Si = pf.sipm_s2_dict(SIPM, S2, thr=50*units.pes)

In [ ]:
plot_pmt_signals(CWFP, signal_start=0, signal_end=1200, ymax = 1)

In [ ]:
plot_signal(csum, signal_start=0, signal_end=1200, ymax = 50)

In [ ]:
plot_signal(csum, signal_start=0, signal_end=590, ymax = 1)

In [ ]:
plot_signal(csum, signal_start=590, signal_end=620, ymax = 150)

In [ ]:
plot_signal(csum, signal_start=620, signal_end=1200, ymax = 2)

1. Before signal: flat tail at about 0.1 pes.
2. After signal: rises to about 1 pes, then decays. 

In [ ]:
S1

In [ ]:
pf.scan_s12(S1)

In [ ]:
pf.scan_s12(S2)

#### Not a krypton signal. 

In [ ]:
pf.plot_s2si_map(S2Si)

A clear track!

### Characterization of the photoelectron tail

1. accumulate csum between 10 and 590
2. accumulate csum between 620 and 1200.
3. plot average

In [ ]:
(1200*units.mus/units.ns)/25

In [ ]:
(550*units.mus/units.ns)/25

In [ ]:
def acc_signal(csum,t0,t1,t2,t3, thr_pre=1*units.pes, thr_post=2*units.pes):
    """1. accumulate csum between 10 and 590
    2. accumulate csum between 620 and 1200.
    
    """
    
    pre = np.zeros((t1 - t0), dtype=np.double)
    sgn = np.zeros((t2 - t1), dtype=np.double)
    post = np.zeros((t3 - t2), dtype=np.double)
    pre = csum[csum < thr_pre][t0:t1]
    post = csum[csum < thr_post][t2:t3]
    sgn = csum[t1:t2]
    return sgn, pre, post

In [ ]:
t0 = 0
t1 = int((550*units.mus/units.ns)/25.) # (1 sample each 25 ns)
t2 = int((650*units.mus/units.ns)/25.)
t3 = int((1200*units.mus/units.ns)/25.)

In [ ]:
sgn, pre, post = acc_signal(csum, t0, t1, t2, t3, thr_pre=1*units.pes, thr_post=2*units.pes)

In [ ]:
plt.plot(sgn)

In [ ]:
sgn.shape[0]

In [ ]:
plot_signal(sgn, signal_start=0, signal_end=100, ymax = 150)

In [ ]:
plt.plot(pre)

In [ ]:
pre.shape[0]/(25)

In [ ]:
plot_signal(pre, signal_start=0, signal_end=550, ymax = 15)

In [ ]:
plt.plot(post)

In [ ]:
post.shape[0]

In [ ]:
plot_signal(post, signal_start=0, signal_end=(1200-650), ymax = 5)

In [ ]:
t0 = 0
t1 = int((590*units.mus/units.ns)/25.) # (1 sample each 25 ns)
t2 = int((620*units.mus/units.ns)/25.)
t3 = int((1190*units.mus/units.ns)/25.)
pre_signal = np.zeros((t1 - t0), dtype=np.double)
post_signal = np.zeros((t3 - t2), dtype=np.double)
signal = np.zeros((t2 - t1), dtype=np.double)

for event in range(NEVT):
    RWF = pmtrwf[event]
    CWF = blr.deconv_pmt(RWF, coeff_c, coeff_blr, n_baseline=38000, thr_trigger=5)
    csum = cpf.calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=100, thr_MAU=3) 
    sgn, pre, post = acc_signal(csum,t0,t1,t2,t3)
    signal += sgn
    pre_signal += pre
    post_signal += post
signal /= event
pre_signal /= event
post_signal /= event

In [ ]:
plt.plot(pre_signal)

In [ ]:
plt.plot(post_signal)

In [ ]:
plt.plot(signal)

In [ ]:
mpl.histo(pre_signal[pre_signal>0.05], nbins=10, 
          title="pre-signal", xlabel="pes", ylabel="Frequency")

In [ ]:
mpl.histo(post_signal[post_signal>0.05], nbins=10, 
          title="post-signal", xlabel="pes", ylabel="Frequency")

In [ ]:
mpl.histo(signal[signal>10], nbins=10, 
          title="signal", xlabel="pes", ylabel="Frequency")

In [ ]:
np.sum(signal)

In [ ]:
np.sum(pre_signal)

In [ ]:
np.sum(post_signal)